Analytics & Visualizations

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # FDA Enforcement Data - Analytics Dashboard
# MAGIC 
# MAGIC Run meaningful analytics on the FDA enforcement recall data.

# COMMAND ----------

from pyspark.sql.functions import *
from pyspark.sql.window import Window

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Overall Recall Statistics

# COMMAND ----------

df_silver = spark.table("fda_enforcement_silver")

total_recalls = df_silver.count()
class_i_count = df_silver.filter(col("classification") == "Class I").count()
class_ii_count = df_silver.filter(col("classification") == "Class II").count()
class_iii_count = df_silver.filter(col("classification") == "Class III").count()
unique_firms = df_silver.select("recalling_firm").distinct().count()
states_affected = df_silver.filter(col("state").isNotNull()).select("state").distinct().count()

print(f"""
FDA Enforcement Recall Statistics
==================================
Total Recalls: {total_recalls:,}
  - Class I (Most Serious): {class_i_count:,} ({class_i_count/total_recalls*100:.1f}%)
  - Class II: {class_ii_count:,} ({class_ii_count/total_recalls*100:.1f}%)
  - Class III (Least Serious): {class_iii_count:,} ({class_iii_count/total_recalls*100:.1f}%)

Unique Recalling Firms: {unique_firms:,}
States Affected: {states_affected}
""")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Recall Trends Over Time

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   year_month,
# MAGIC   classification,
# MAGIC   recall_count
# MAGIC FROM fda_recalls_monthly_trends
# MAGIC ORDER BY year_month, classification

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Top 20 Recalling Firms (All Classifications)

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   recalling_firm,
# MAGIC   classification,
# MAGIC   total_recalls,
# MAGIC   states_count as states_affected,
# MAGIC   first_recall,
# MAGIC   most_recent_recall
# MAGIC FROM fda_recalls_top_firms
# MAGIC ORDER BY total_recalls DESC
# MAGIC LIMIT 20

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. State-by-State Recall Analysis

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   state,
# MAGIC   SUM(total_recalls) as total_recalls,
# MAGIC   SUM(CASE WHEN classification = 'Class I' THEN total_recalls ELSE 0 END) as class_i,
# MAGIC   SUM(CASE WHEN classification = 'Class II' THEN total_recalls ELSE 0 END) as class_ii,
# MAGIC   SUM(CASE WHEN classification = 'Class III' THEN total_recalls ELSE 0 END) as class_iii,
# MAGIC   SUM(unique_firms) as total_firms
# MAGIC FROM fda_recalls_summary_state
# MAGIC GROUP BY state
# MAGIC ORDER BY total_recalls DESC
# MAGIC LIMIT 20

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. California-Specific Analysis

# COMMAND ----------

df_ca = df_silver.filter(
    (col("state") == "CA") | (col("distribution_pattern").contains("CA"))
)

print(f"California Recalls: {df_ca.count():,}")
print("\nBreakdown by Classification:")
df_ca.groupBy("classification").count().orderBy("count", ascending=False).show()

print("\nTop 10 Firms with CA Recalls:")
display(
    df_ca.groupBy("recalling_firm", "classification")
    .count()
    .orderBy("count", ascending=False)
    .limit(10)
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. Recent Class I Recalls (High Priority)

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   report_date,
# MAGIC   recalling_firm,
# MAGIC   product_description,
# MAGIC   reason_for_recall,
# MAGIC   state,
# MAGIC   distribution_pattern,
# MAGIC   status
# MAGIC FROM fda_recalls_class_i
# MAGIC ORDER BY report_date DESC
# MAGIC LIMIT 50

# COMMAND ----------

# MAGIC %md
# MAGIC ## 7. Voluntary vs Mandated Recalls

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   voluntary_mandated,
# MAGIC   classification,
# MAGIC   COUNT(*) as recall_count,
# MAGIC   COUNT(DISTINCT recalling_firm) as unique_firms
# MAGIC FROM fda_enforcement_silver
# MAGIC WHERE voluntary_mandated IS NOT NULL
# MAGIC GROUP BY voluntary_mandated, classification
# MAGIC ORDER BY voluntary_mandated, classification

# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Product Type Distribution

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   product_type,
# MAGIC   total_recalls,
# MAGIC   unique_firms,
# MAGIC   states_affected
# MAGIC FROM fda_recalls_product_analysis
# MAGIC ORDER BY total_recalls DESC

# COMMAND ----------

# MAGIC %md
# MAGIC ## 9. Most Common Recall Reasons (Text Analysis)

# COMMAND ----------

# Get top recall reasons
df_reasons = df_silver \
    .filter(col("reason_for_recall").isNotNull()) \
    .groupBy("reason_for_recall", "classification") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(30)

print("Top 30 Recall Reasons:")
display(df_reasons)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 10. Recall Status Distribution

# COMMAND ----------

# MAGIC %sql
# MAGIC SELECT 
# MAGIC   status,
# MAGIC   classification,
# MAGIC   COUNT(*) as recall_count,
# MAGIC   ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
# MAGIC FROM fda_enforcement_silver
# MAGIC WHERE status IS NOT NULL
# MAGIC GROUP BY status, classification
# MAGIC ORDER BY recall_count DESC

# COMMAND ----------

# MAGIC %md
# MAGIC ## 11. Geographic Distribution Analysis

# COMMAND ----------

# Countries with most recalls
print("Top Countries by Recall Count:")
display(
    df_silver
    .filter(col("country").isNotNull())
    .groupBy("country", "classification")
    .count()
    .orderBy("count", ascending=False)
    .limit(20)
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 12. OpenFDA Device Information Analysis

# COMMAND ----------

# Analyze device classes and specialties
df_devices = df_silver \
    .filter(col("openfda").isNotNull()) \
    .select(
        "classification",
        "openfda.device_name",
        "openfda.device_class",
        "openfda.medical_specialty_description"
    )

print("Device Classes Distribution:")
display(
    df_devices
    .groupBy("device_class", "classification")
    .count()
    .orderBy("count", ascending=False)
)

print("\nMedical Specialties:")
display(
    df_devices
    .filter(col("medical_specialty_description").isNotNull())
    .groupBy("medical_specialty_description")
    .count()
    .orderBy("count", ascending=False)
    .limit(20)
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 13. Custom Analysis: High-Risk Patterns

# COMMAND ----------

# Find firms with multiple Class I recalls
high_risk_firms = df_silver \
    .filter(col("classification") == "Class I") \
    .groupBy("recalling_firm") \
    .agg(
        count("*").alias("class_i_recalls"),
        collect_set("reason_for_recall").alias("recall_reasons"),
        min("report_date").alias("first_class_i"),
        max("report_date").alias("most_recent_class_i")
    ) \
    .filter(col("class_i_recalls") >= 2) \
    .orderBy("class_i_recalls", ascending=False)

print("Firms with Multiple Class I Recalls:")
display(high_risk_firms)


FDA Enforcement Recall Statistics
Total Recalls: 3,503
  - Class I (Most Serious): 400 (11.4%)
  - Class II: 3,070 (87.6%)
  - Class III (Least Serious): 32 (0.9%)

Unique Recalling Firms: 472
States Affected: 38

California Recalls: 959

Breakdown by Classification:
+------------------+-----+
|    classification|count|
+------------------+-----+
|          Class II|  852|
|           Class I|   95|
|         Class III|   11|
|Not Yet Classified|    1|
+------------------+-----+


Top 10 Firms with CA Recalls:


recalling_firm,classification,count
"Jiangsu Shenli Medical Production Co., Ltd.",Class II,87
Atrium Medical Corporation,Class II,68
"Merit Medical Systems, Inc.",Class II,47
"CareFusion 303, Inc.",Class II,40
Olympus Corporation of the Americas,Class II,25
"MEDLINE INDUSTRIES, LP - Northfield",Class II,25
Medtronic Inc.,Class II,23
Abbott Medical,Class II,19
"Angiodynamics, Inc.",Class II,16
The Metrix Company,Class II,16


Top 30 Recall Reasons:


reason_for_recall,classification,count
"Medical convenience kits contain plastic syringes affected by the FDA Safety Alert issued on March 19, 2024. Leaks, breakage, and/ other quality issues have been identified that may pose a risk to patient health.",Class II,160
"Due to a voluntary recall of low dead space syringes, and luer slip tip syringes manufactured by Sol-Millennium Medical (Sol-M), Inc. These syringes manufactured by Sol-M are present in some of BVI CustomEyes Procedure Packs.",Class II,158
Piston syringes sizes and configurations are out of the range of devices cleared under the firm's 510(k).,Class II,87
Sponge forceps and towel clamps associated with small loose metal flakes potentially could detach from the components. Metal flakes on components may enter a patients surgical site undetected causing local reactions or possible foreign body reactions,Class II,73
Complaints of reported separation of the Slider GDS Swivel Rod from the Swivel Core and of a notable gap between the two pieces.,Class II,68
Product manufactured using UHMWPE raw material over 5 years of age has the potential for elevated levels of oxidation. Oxidation within UHMWPE can have an impact on its material properties.,Class II,58
"Medline Industries, LP is recalling certain kits and trays that were manufactured using specific lots of Nurse Assist (as components): 0.9% Sodium Chloride Irrigation USP, Sterile Water for Irrigation USP and Saline Flush Syringe. Nurse Assist issued a recall due to the potential lack of sterility, which could result in the solution being non sterile.",Class I,49
An increase in complaints related to an inability to advance the guidewire through the lumen of the device for certain batches. There is evidence of the polyurethane layer delaminating and in some instances material detachment in the catheters inner lining.,Class I,48
Patient support table floor plate may be incorrectly installed.,Class II,41
Ethylene Oxide/Ethylene Chlorohydrin (EO/ECH) residuals for the cast padding component exceeded the residual limits for permanent exposure devices as indicated by ANSI/AAMI/ISO 10993-7/(R)2012.,Class II,39


Top Countries by Recall Count:


...

device_class,classification,count
null,Class II,3070
null,Class I,400
null,Class III,32
null,Not Yet Classified,1



Medical Specialties:


medical_specialty_description,count


Firms with Multiple Class I Recalls:


recalling_firm class_i_recalls recall_reasons first_class_i most_recent_class_i Boston Scientific Corporation 57 List(An increase in complaints related to an inability to advance the guidewire through the lumen of the device for certain batches. There is evidence of the polyurethane layer delaminating and in some instances material detachment in the catheters inner lining., Boston Scientific is updating the instructions for use of their POLARx and POLARx FIT Cryoablation Balloon Catheters related to Atrio-esophageal Fistula Risk., An investigation determined that delivery of the Obsidio embolic using the aliquot technique for lower gastrointestinal bleeding embolization poses a high risk of bowel ischemia. The most serious and the most common adverse health consequence, reasonably foreseeable to occur, is the need to perform major surgery such as bowel resection and/or diverting colostomy. Therefore, Boston Scientific does not recommend that the aliquot technique be used to deliver the Obsidio device for lower GI bleed embolization procedures., Boston Scientific has identified a tooling error in manufacturing which may have caused delamination of the inner lumen of the sheath shaft in a subset of POLARSHEATH devices. This could result in the embolization of a fragment during use (such as flushing of the sheath or introduction of the dilator or ablation catheter).) 20240403 20241120 MEDLINE INDUSTRIES, LP - Northfield 56 List(Medline Industries, LP is recalling certain kits and trays that were manufactured using specific lots of Nurse Assist (as components): 0.9% Sodium Chloride Irrigation USP, Sterile Water for Irrigation USP and Saline Flush Syringe. Nurse Assist issued a recall due to the potential lack of sterility, which could result in the solution being non sterile., Medical convenience kits contain plastic syringes affected by the FDA Safety Alert issued on March 19, 2024. Leaks, breakage, and/ other quality issues have been identified that may pose a risk to patient health., Complaints have been received that the device inflation tube detached and/or tore from the main tube, resulting in potential moisture buildup, loss of pressure, or inability to inflate. There were also reports the suction pump is difficult to connect or detaches during use., The component tracheostomy brush has a sharp edge at the tip that can potentially puncture tubing during use and/or cause user injury. Additionally, there have been incidents of the tracheostomy brush bristles detaching prior to use in the packaging and/or during use.) 20240103 20240814 Smiths Medical ASD Inc. 43 List(Smiths Medical has identified that the securement flange of specific lots of the Bivona Neonatal/Pediatric and Adult Tracheostomy products may tear because of a manufacturing defect., The CADD Solis VIP Ambulatory Infusion Pump is indicated for the following uses: " For intravenous, intraarterial, subcutaneous, intraperitoneal, perineural, surgical site, epidural space, or subarachnoid space infusion., Medfusion syringe pumps, with software versions before v6.0.0, may have the following issues:1. Delivery During Motor Not Running High Priority Alarm, 2. Infusion Restarted with Incorrect Parameters, 3. Screen Lock, 4. Interruption of Bolus or Loading Dose Delivery, 5. Pump Displays Incorrect Bolus/Loading Dose, 6. Loading/Bolus Dose Below the Minimum Recommended Rate, 7. Motor Rate Error, 8. Incorrect Recall Last Settings, 9. Corrupt Configuration, 10. Auto Lock, 11. Toolbox Configuration Loading Dose Time Values. Smiths Medical corrected all issues included in this notification in previous software updates and the corrections were carried forward into all subsequent software releases. Please ensure you have the most recent Medfusion software (v6.0.0) installed on your pumps., When a paraPAC plus ventilator is switched to the operating mode of VENTILATE, the ventilator may intermittently provide continuous positive gas flow instead of the intended cycling like a hu